In [1]:
import matplotlib.pyplot as plt
import numpy as np
# import torch
import pickle
# from torch import nn, optim

from models import TfIdf, SentimentPredictorTfIdf
from utils import DataLoader
from IPython.display import clear_output

In [2]:
BATCH_SIZE = 128
EPOCHS = 10
ETA = 1e-3
NUM_WORDS = 30185
NUM_DOCS = 18000

In [3]:
tfidf = TfIdf(num_words=NUM_WORDS, num_docs=NUM_DOCS)

In [4]:
# dloader = DataLoader(10)
train_loader = DataLoader(BATCH_SIZE, mode="train")
valid_loader = DataLoader(BATCH_SIZE, mode="valid")
test_loader = DataLoader(BATCH_SIZE, mode="test")

In [ ]:

# def drain_queue():
#     while True:
        

In [5]:
def train_tf_idf():
    
    for x, y in train_loader.next_batch():
        for doc in x:
            tfidf.fit(doc)
            
    tfidf.save("ckpts/tfidf2.bin")

In [6]:
# train_tf_idf()
tfidf.load("ckpts/tfidf2.bin")

In [7]:
def generate_tf_idf():
    
    i = 0
    ys = []
    for x, y in train_loader:
        if i == 19395:
            break
#         with open(f"../../../data/moneyfin/indexing/{i}_indexing.bin", "wb") as f:
#             f.write(pickle.dumps(x))
#         doc = tfidf.predict(x)
#         with open(f"../../../data/moneyfin/indexing/{i}_tfidf.bin", "wb") as f:
#             f.write(pickle.dumps(doc))
            
        ys.append(y)
        i += 1
        
    with open(f"../../../data/moneyfin/indexing/y.bin", "wb") as f:
        f.write(pickle.dumps(ys))

In [8]:
generate_tf_idf()

IndexError: list index out of range

In [9]:
clf = SentimentPredictorTfIdf().cuda()
optimizer = optim.Adam(clf.parameters(), lr=ETA)
criterion = nn.BCELoss()

In [10]:
def compute_accuracy(preds, y):
    preds[preds < 0.5] = 0
    preds[preds >= 0.5] = 1
    equals = preds == y
    acc = torch.mean(equals.type(torch.FloatTensor))
    return acc

In [21]:
def train_sentiment_model():
    
    train_losses = []
    valid_losses = []
    
    for e in range(EPOCHS):
        train_loss = 0
        train_acc = 0
        valid_loss = 0
        valid_acc = 0
        
        clf.train()
        
        for x, y in train_loader.next_batch():
            x_tensor = torch.zeros(len(x), NUM_WORDS).cuda()
            y = torch.FloatTensor(y).cuda()
            
            for i, doc in enumerate(x):
                x_tensor[i] = torch.FloatTensor(tfidf.predict(doc)).cuda()
                
            preds = clf(x_tensor)
            loss_0 = criterion(preds[y == 0].view(-1), y[y == 0])
            loss_1 = criterion(preds[y == 1].view(-1), y[y == 1])
            loss = (5/6)*loss_0 + (1/6)*loss_1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_acc += compute_accuracy(preds, y)
            
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        
        clf.eval()
            
        with torch.zero_grad():
            for x, y in valid_loader.next_batch():
                x_tensor = torch.zeros(len(x), NUM_WORDS).cuda()
                y = torch.FloatTensor(y).cuda()

                for i, doc in enumerate(x):
                    x_tensor[i] = torch.FloatTensor(tfidf.predict(doc)).cuda()

                preds = clf(x_tensor)
                loss_0 = criterion(preds[y == 0].view(-1), y[y == 0])
                loss_1 = criterion(preds[y == 1].view(-1), y[y == 1])
                loss = (5/6)*loss_0 + (1/6)*loss_1
                
                valid_loss += loss.item()
                valid_acc += compute_accuracy(preds, y)
            
            valid_loss /= len(valid_loader)
            valid_acc /= len(valid_loader)
            
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        clear_output(wait=True)
            
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.6f}, train acc: {train_acc:.4f}")
        print(f"Valid loss: {valid_loss:.6f}, valid acc: {valid_acc:.4f}")
        
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        
        axes[0].plot(train_losses)
        axes[0].set_title("train loss")
        axes[0].set_xlabel("epochs")
        axes[0].set_ylabel("loss")
        
        axes[1].plot(valid_losses)
        axes[1].set_title("valid loss")
        axes[1].set_xlabel("epochs")
        axes[1].set_ylabel("loss")
        
        plt.show()

In [22]:
train_sentiment_model()

KeyboardInterrupt: 